In [1]:
from __future__ import annotations

import json

from enum import Enum
from dataclasses import dataclass, field
from typing import List, Any

from shapely.geometry.base import BaseGeometry
from shapely import Polygon, Point, MultiPolygon

In [2]:
@dataclass
class Area:
    polygon: Polygon = field(repr=False, hash=False)
    name: str
    value_Object: Any = field(default=None)

    def intersects(self, other: BaseGeometry) -> bool:
        return self.polygon.intersects(other)

    def touches(self, other: BaseGeometry) -> bool:
        return self.polygon.touches(other)

    def touches_or_intersects(self, other: BaseGeometry) -> IntersectsTouchesResult:
        response: False | IntersectsTouchesResultTypeEnum = False

        if self.intersects(other):
            response = IntersectsTouchesResultTypeEnum.intersects

        if self.touches(other):
            response = IntersectsTouchesResultTypeEnum.touches

        if response:
            return IntersectsTouchesResult(self, response)

        return IntersectsTouchesResult(self, IntersectsTouchesResultTypeEnum.no_match)


@dataclass
class IntersectsTouchesResult:
    area: Area
    result: IntersectsTouchesResultTypeEnum


class IntersectsTouchesResultTypeEnum(Enum):
    intersects = "intersects"
    touches = "touches"
    no_match = "no match"


@dataclass
class AreaClassifierResult:
    geometry: BaseGeometry
    _areas: List[Area | IntersectsTouchesResult]
    _distance: List[List[str, float]]
    matches: List[IntersectsTouchesResult] = field(init=False)
    neighbours: List[Area, float] = field(init=False)

    def __post_init__(self):
        self.matches = self.areas()
        # todo should get all intersect or touches, this does not work ;)
        self.neighbours = [item for item in self.within_distance(20) if item not in self.matches]

    def areas(self, only_match=True) -> List[IntersectsTouchesResult]:
        if only_match:
            return [item for item in self._areas if item.result != IntersectsTouchesResultTypeEnum.no_match]
        return [item for item in self._areas]

    def within_distance(self, search_range = 25_000) -> List[Area, float]:
        # todo make zip as object
        return [item for item in list(zip(self._areas, self._distance)) if item[1] < search_range]


class AreaClassifier:
    def __init__(self, areas: List[Area]):
        self._areas: List[Area] = areas

    def _determine_areas(self, geometry: BaseGeometry) -> list[IntersectsTouchesResult]:
        return [area.touches_or_intersects(geometry) for area in self._areas]

    def _determine_distance(self, geometry: BaseGeometry) -> List[List[str, float]]:
        return [area.polygon.boundary.distance(geometry) for area in self._areas]

    def classify(self, geometry:BaseGeometry):
        return AreaClassifierResult(
            geometry=geometry,
            _areas=self._determine_areas(geometry),
            _distance=self._determine_distance(geometry),
        )


@dataclass
class BedienGebied:
    _data: dict
    naam: str = field(init=False)
    afkorting: str = field(init=False)
    soort: str = field(init=False)
    weergave: str = field(init=False)

    def __post_init__(self):
        self.naam = self._data['NAAM']
        self.afkorting = self._data['NAAM_AFKORTING']
        self.soort = self._data['SOORT']
        self.weergave = self._data['SOORT_NAAM_AFKORTING']

def get_vl_post_areas() -> List[Area]:
    with open('bediengebied.json') as f:
        vl_post_json = json.loads(f.read())

    return  [
        Area(
            MultiPolygon([Polygon(item2) for item2 in item['geometry']['rings']]),
            item['attributes']['SOORT_NAAM_AFKORTING'],
            BedienGebied(item['attributes'])
        ) for item in
        vl_post_json['features']
    ]


In [3]:
classifier = AreaClassifier(get_vl_post_areas())

In [4]:
from imxInsights import Imx

In [5]:
file_path = r"..\\..\sample_data\Hanzelijn_verrijkt_latest.xml"
file_path = r"C:\Users\Hzd\Downloads\IMX_E-R50009_EHL_Perceel_2_C1.0_5_0_0_test.xml"
imx = Imx(file_path)

In [6]:
results = []

for item in imx.project.new_situation.get_all():
    results.append([
        item.puic, 
        [item2.area.value_Object for item2 in classifier.classify(item.shapely).matches]
    ])

C:\ProRail\Python\imxInsightsMonoRepo\.venv\lib\site-packages\shapely\predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
C:\ProRail\Python\imxInsightsMonoRepo\.venv\lib\site-packages\shapely\predicates.py:894: RuntimeWarning: invalid value encountered in touches
  return lib.touches(a, b, **kwargs)


In [7]:
import pprint as pp

In [8]:
out_list = []
for item in results:
    out_list.append([item[0], " & ".join([F"{item2.naam} ({item2.soort})" for item2 in item[1]])])

In [9]:
import pandas as pd
df = pd.DataFrame(out_list, columns = ['puic', 'bedien gebied'])
df

,puic,bedien gebied
0,7815ba93-2f6f-493b-a0f5-175a7868b438,Almere Oostvaarders (PPLG)
1,4d7f4f7e-3ae0-45ae-a3cb-9e51df4ab6ac,Almere Oostvaarders (PPLG)
2,d240e241-bd46-4c2a-8514-67e15d6b2702,Almere Oostvaarders (PPLG)
3,24a90a09-94a7-45a6-b86d-dd3e52cb86d9,Almere Oostvaarders (PPLG)
4,863171af-934e-459f-a3a1-bd6b8cbd4645,Almere Oostvaarders (PPLG)
...,...,...
6323,e8b36985-72e2-406f-be13-270e07854d20_ErtmsBali...,
6324,88042cb3-4663-425c-9bc2-1d5f4fa39244_ErtmsBali...,
6325,039d184d-1d42-40d9-8713-1944723a7425_ErtmsBali...,
6326,f392dd9f-1706-4161-872c-8749144dcfa7_ErtmsBali...,


In [23]:
df.to_excel("tester.xlsx")